In [1]:
from abi import Abi
import pandas as pd
from _const import * # paths 
from _func import * # lambdas

In [2]:
abi_obj = Abi(PATH_ABIS.ls()[0])

In [3]:
df_events = abi_obj.create_events_df()

In [4]:

df_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 16
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          16 non-null     object
 1   types         16 non-null     object
 2   names         16 non-null     object
 3   indexed       16 non-null     object
 4   types_data    16 non-null     object
 5   types_topics  16 non-null     object
 6   signature     16 non-null     object
 7   hash          16 non-null     object
dtypes: object(8)
memory usage: 1.1+ KB


In [5]:
types_data_list = []
types_topics_list = []

for _, x in df_events.iterrows():
    tmp_list = list(zip(x['types'], x['indexed']))
    types_topics = []
    types_data = []
    for arg_type, indexed in tmp_list:    
        if indexed:
            types_topics.append(arg_type)
        else:
            types_data.append(arg_type)

    if not types_topics: types_topics = None
    if not types_data: types_data = None
    types_topics_list.append(types_topics)
    types_data_list.append(types_data)

df_events['types_topics'] = types_topics_list
df_events['types_data'] = types_data_list



In [19]:
[(t, i) for t, i in zip(x['types'], x['indexed']) if i]

[('address', True)]

In [92]:
import uuid
str(uuid.uuid4())


'c35677fb-66e3-403b-b888-4c5cb91c4d0b'

In [ ]:
[{'name': name, 'job': job, 'status': status} for name,job,status in zip(names,jobs,statuses)]


In [6]:
df_events.head()

,name,types,names,indexed,types_data,types_topics,signature,hash
1,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
2,GuardianActivationLockChanged,"[address, address, uint256, uint256]","[guardian, lockManager, amount, total]","[True, True, False, False]","[uint256, uint256]","[address, address]","GuardianActivationLockChanged(address,address,...",0xb754cf0e7e51fa0b0a5ed45913a6da3e3aa291da6250...
3,GuardianBalanceLocked,"[address, uint256]","[guardian, amount]","[True, False]",[uint256],[address],"GuardianBalanceLocked(address,uint256)",0xc3d70b7809835f6deb56c03ebabf6aecdf78e9b4da7c...
4,GuardianBalanceUnlocked,"[address, uint256]","[guardian, amount]","[True, False]",[uint256],[address],"GuardianBalanceUnlocked(address,uint256)",0x389bab47d019f9160c099e9beca71df98fb81c079bfb...
5,GuardianDeactivationProcessed,"[address, uint64, uint256, uint64]","[guardian, availableTermId, amount, processedT...","[True, False, False, False]","[uint64, uint256, uint64]",[address],"GuardianDeactivationProcessed(address,uint64,u...",0xb7ae268c06bce2729709d55ad102f434a28f71877bfa...


In [7]:
from bq import get_logs
df_logs = get_logs(address=[ADDRESS_COURT], func_sign=[FUNC_SIGN_GUARDIANS_ACTIVATED])


            SELECT * 
            FROM `bigquery-public-data.crypto_ethereum.logs`, 
            UNNEST(topics) t 
            WHERE 
            t in (  "0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116b141b1a1a2e8f0404095"   )
            AND address in ( "0xab647b8fd9e370448d4eeb96582fe839f3d0bb24" ) 
            AND DATE(block_timestamp) >= "2020-01-01"
            LIMIT 100


In [8]:
df_logs = df_logs.merge(df_events, left_on='t', right_on='hash')

In [9]:
df_logs.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,t,name,types,names,indexed,types_data,types_topics,signature,hash
0,41,0x068e26ee5c224fd4a1d7c93f04a0727fe4148bbc19f6...,45,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-15 09:30:41+00:00,13028933,0x9f7259689fabecf310101de69c40a05d471bff895234...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
1,105,0x25050da24070b78cbf8e4d21cc5addfc5ec74a3cc5b2...,61,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-15 07:34:37+00:00,13028413,0xaab386dbcac469094587cef9949a23de2f5996ba789e...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
2,181,0x0ca00eab6811c68301af1990ff6a2bd5a2947ca415d9...,77,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-15 06:42:42+00:00,13028204,0xcc1ce0f49954def1e170059f02522c4fc0a20a6af412...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
3,162,0x0bc02bd40a6bd30b72f591d6e7fc7e3fbf70bc92fc32...,115,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-15 03:25:39+00:00,13027345,0x12d1fd085cd0e6aadd4e3e2c0ef70288e6977c754b5d...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
4,387,0x5ad90c38f722b9562674b9946b41752890834fd6b122...,173,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-12 04:24:03+00:00,13008169,0x33cf8f089c9a15f42651952de052dc202733170fdf7d...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...


In [10]:
def decodee(x):
    try:
        return decode_abi(x['types_data'], str.encode(x['data']))
    except Exception as e:
        return e

In [11]:
from eth_utils import decode_hex


lambda_decode_data = lambda x: decode_abi(x['types_data'], decode_hex(x['data']))
df_logs.apply(decodee, axis=1).value_counts()

Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
                                                            ..
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Length: 99, dtype: int64

In [ ]:
web3.eth.abi.decodeParameters

In [17]:
i = 1
print(df_logs['types_data'][i])
print(df_logs['data'][i])

['uint64', 'uint256']
0x00000000000000000000000000000000000000000000000000000000000000ba00000000000000000000000000000000000000000000001c9f78d2893e400000


In [60]:
from google.cloud import bigquery
big_query_client = bigquery.Client()

table_ref = big_query_client.dataset('base_information').table('test')
table = big_query_client.get_table(table_ref)  # API request

In [61]:
table.schema

[SchemaField('id', 'STRING', 'NULLABLE', None, (), ())]

In [95]:
a = big_query_client.insert_rows(table, [{'id':1, 'time':2}])

In [97]:
type(a)

list

In [99]:
import requests

URL = 'https://api.thegraph.com/subgraphs/name/aragon/aragon-court-v2-mainnet'

r = requests.get(URL)

print(r)

<Response [200]>


In [127]:

query = """{
  courts(first: 5) {
    id
    token {
      id
    }
    currentTerm
    termDuration
  }
  
}"""

In [128]:
import json
r = requests.post(URL, json={'query': query})
print(r.status_code)
print(r.text)
json_data = json.loads(r.text)




200
{"data":{"courts":[{"currentTerm":"189","id":"0xfb072baa713b01ce944a0515c3e1e98170977daf","termDuration":"28800","token":{"id":"0xa117000000f279d81a1d3cc75430faa017fa5a2e"}}]}}


In [130]:
json_data['data']

{'courts': [{'currentTerm': '189',
   'id': '0xfb072baa713b01ce944a0515c3e1e98170977daf',
   'termDuration': '28800',
   'token': {'id': '0xa117000000f279d81a1d3cc75430faa017fa5a2e'}}]}